In [1]:
import numpy as np
import tensorflow as tf
import cv2
from ultralytics import YOLO
from norfair import Detection, Tracker
import uuid


In [12]:
video_path = "../data/people_walking.mp4" #video path

 #"n" so it is nano model, faster. 
 #It is trained by COCO dataset that includes 80 different labels 
 #I used nano model but you can use small, medium,...
model = YOLO("../yolov8n.pt")

In [13]:
interpreter = tf.lite.Interpreter(model_path="../reid_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [14]:
def get_embedding(image):
    image = cv2.resize(image, (128,256))
    image = image / 255.0
    image = np.expand_dims(image.astype(np.float32), axis=0)
    interpreter.set_tensor((input_details[0]['index']), image)
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])[0]

In [15]:
def cosine_similarity(a, b):
    return np.dot(a, b) /(np.linalg.norm(a) * np.linalg.norm(b))

In [21]:
def generate_new_id():
    return str(uuid.uuid4())[:8]

In [16]:
tracker = Tracker(distance_function="euclidean", distance_threshold=25)

In [20]:
cap = cv2.VideoCapture(video_path)

if not cap.isOpened:
    print("Video could not open")
    exit()

unique_ids = set()
reid_db = {} #track_id : embedding


In [ ]:
while True:
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame)[0]
    detections = []
    new_reid_matches = {} # detectionindex : matched_id

    for i, box in enumerate(results.box):
        cls_id = int(box.cls[0])

    if model.names[cls_id] == "person":
        conf = float(box.conf[0])

        if conf > 70:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = int((x1 + x2) / 2), int((y1+ y2) /2)

            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue

            embedding = get_embedding(person_crop)
            matched_id = None

            for known_id, known_embedding in reid_db.items():
                if cosine_similarity(embedding, known_embedding) > 0.70:
                    matched_id = known_id
                    break

            if matched_id is None:
                matched_id = generate_new_id()
                reid_db[matched_id] = embedding
            else: # Burayla sonradan oynayabilirim.
                reid_db[matched_id] = embedding 

            
